### **(c) Gibbs Sampler**

To use the Gibbs sampler we first need to derive the conditional distribution of each variable given the others. So given $P(i,j)$ from above we need to derive $P(i|j)$ and $P(j|i)$. First we derive $P(i|j)$:

\begin{align*}
    P(i|j) &= \frac{P(i,j)}{P(j)} = \frac{P(i,j)}{\sum_{i} P(i,j)} \\
    &= \frac{P(i,j)}{\frac{A_2^j}{j!}\sum_{i}\frac{A_1^i}{i!}} = \frac{P(i,j)}{\frac{A_2^j}{j!}ce^{A_1}} \\
    &= \frac{c\frac{A_1^i}{i!}\frac{A_2^j}{j!}}{\frac{A_2^j}{j!}ce^{A_1}} = \frac{A_1^i}{e^{A_1}} = \frac{A_1^i e^{-A_1}}{i!} = \text{Pois}(A_1)
\end{align*}

From the derivation above we see that the conditional distribution is a Poisson distribution: $P(i|j) = \text{Pois}(A_1)$. It is possible to do the same for $P(j|i)$ and we will find that $P(j|i) = \text{Pois}(A_2)$. This means that we can use the Gibbs sampler to sample from the posterior distribution of $i$ and $j$ given the other variable.

In [56]:
import numpy as np
import math
import plotly.graph_objects as go

In [67]:
def p_i_give_j(i,j,A1,m):
    numerator = (A1**i)/math.factorial(i)

    #factorials = [math.factorial(i) for i in range(m-j+1)]
    denominator = np.sum([(A1**k)/math.factorial(k) for k in range(m-j+1)])

    return numerator/denominator

def p_j_give_i(i,j,A2,m):
    numerator = (A2**j)/math.factorial(j)

    #factorials = [math.factorial(j) for j in range(m-i+1)]
    denominator = np.sum([(A2**k)/math.factorial(k) for k in range(m-i+1)])

    return numerator/denominator

def gibbs_sampler(n, x0, A1, A2, m):
    # Handle if x0 is a scalar
    x0 = np.array([x0]) if np.shape(x0) == () else x0

    X = np.zeros((n+1, len(x0)))
    X[0] = x0

    for k in range(n):
        i,j = X[k]

        i = int(i)
        j = int(j)

        i = np.random.choice(np.arange(0,m-j+1), p=[p_i_give_j(h,j,A1,m) for h in range(m-j+1)])
        j = np.random.choice(np.arange(0,m-i+1), p=[p_j_give_i(i,h,A2,m) for h in range(m-i+1)])

        Y = np.array([i,j])
        X[k+1] = Y
                
    return X

In [70]:
A1 = 4
A2 = 4
m = 10

n = 10000
i = np.random.randint(0,m+1)
j = np.random.randint(0,m-i+1)
x0 = np.array([i,j])

samples = gibbs_sampler(n, x0, A1, A2, m)

fig = go.Figure(data=[go.Histogram2d(x=samples[:,0], y=samples[:,1], colorscale='Viridis')])
fig.update_layout(title='Joint histogram of i and j', xaxis_title='i', yaxis_title='j')
fig.show()

## **Part 3 - Bayesian Statistical Problem**

In [71]:
def f(x,y,rho=1/2):

    numerator = -np.log(x)**2-2*rho*np.log(x)*np.log(y) + np.log(y)**2
    denominator = 2*(1-rho**2)

    mul1 = 1/(2*np.pi*x*y*np.sqrt(1-rho**2))
    mul2 = numerator/denominator

    return mul1*mul2

In [73]:
def metropolis_hastings(g, proposal_sampler, n, x0):

    # Handle if x0 is a scalar
    x0 = np.array([x0]) if np.shape(x0) == () else x0

    X = np.zeros((n+1, len(x0)))
    X[0] = x0

    for i in range(n):
        # Propose a new state
        Y = proposal_sampler(X[i])
        Y = np.array([Y]) if np.shape(Y) == () else Y

        # Evaluate the target distribution
        g_Y = g(Y)
        g_X = g(X[i])

        # Accept or reject the new state
        if g_Y >= g_X:
            X[i+1] = Y
        else:
            accept = np.random.uniform(0,1) < g_Y/g_X
            X[i+1] = Y if accept else X[i]
                
    return X

In [80]:
mean = [0, 0]
cov = [[1, 0.5], [0.5, 1]]

# Generate pairs of correlated normal variables
x, y = np.random.multivariate_normal(mean, cov).T

# Exponentiate the variables to get log-normal variables
theta = np.exp(x)
phi = np.exp(y)

num_samples = 1000

proposal_sampler = lambda x: np.random.multivariate_normal(x, [[0.1, 0], [0, 0.1]])

samples = metropolis_hastings(lambda x: f(x[0], x[1]), proposal_sampler, num_samples, [theta, phi])

# make trace plots for theta and phi
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(num_samples+1), y=samples[:,0], mode='lines', name='Theta'))
fig.add_trace(go.Scatter(x=np.arange(num_samples+1), y=samples[:,1], mode='lines', name='Phi'))
fig.update_layout(title='Trace plots for Theta and Phi', xaxis_title='Iteration', yaxis_title='Value')
fig.show()

/tmp/ipykernel_8802/2107627643.py:3: RuntimeWarning:

invalid value encountered in log

